In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import itertools

In [2]:
# Define the name of your Excel file.
excel_filename = 'population_hhld.xlsx'

# Read the 'data' and 'criteria' sheets directly from the Excel file.
# Using a try-except block to handle potential file/sheet not found errors.
try:
    data = pd.read_excel(excel_filename, sheet_name='data')
    criteria = pd.read_excel(excel_filename, sheet_name='criteria')
    print(f"Successfully read 'data' and 'criteria' sheets from '{excel_filename}'.")
except FileNotFoundError:
    print(f"Error: Make sure the Excel file ('{excel_filename}') is in the same directory as the script.")
    exit()
except ValueError as e:
    print(f"Error: Could not find a sheet named 'data' or 'criteria' in the Excel file. Details: {e}")
    exit()

Successfully read 'data' and 'criteria' sheets from 'population_hhld.xlsx'.


In [ ]:
# --- Part 2: Data Processing and Transformation ---

In [ ]:
# 2.1: Create the ind_cntry_yr DataFrame

# First, clean the 'Year' column in the original data by removing the 'Y' prefix and converting it to a numeric type.
# We use .loc to avoid SettingWithCopyWarning.
data.loc[:, 'Year'] = data['Year'].astype(str).str.replace('Y', '').astype(int)

# Define the full range of years we are interested in (2010-2025).
years_range = list(range(2010, 2026))

# Get unique indicators and countries from the data.
unique_indicators = data['Indicator'].unique()
unique_countries = data['Country'].unique()

# Create a master list of all possible combinations of Indicator, Country, and Year.
# This forms a complete grid with the empty years too to check data availability against.
all_combinations = list(itertools.product(unique_indicators, unique_countries, years_range))
ind_cntry_yr = pd.DataFrame(all_combinations, columns=['Indicator', 'Country', 'Year'])

# Merge the complete grid with the actual data.
# A 'left' merge keeps every combination from our master list.
# The 'indicator' tells us if a match was found in the original data.
ind_cntry_yr = pd.merge(
    ind_cntry_yr,
    data[['Indicator', 'Country', 'Year', 'Value']],
    on=['Indicator', 'Country', 'Year'],
    how='left'
)

# Create the 'Valid' column. If 'Value' is not NaN (meaning data was present),
# mark it as 1 (available), otherwise mark it as 0 (unavailable).
ind_cntry_yr['Valid'] = ind_cntry_yr['Value'].notna().astype(int)

# We no longer need the 'Value' column for this analysis.
ind_cntry_yr = ind_cntry_yr.drop(columns=['Value'])

# Save the resulting DataFrame to an Excel file.
ind_cntry_yr.to_excel("ind_cntry_yr.xlsx", index=False)
print("Saved 'ind_cntry_yr.xlsx'")

In [ ]:
# 2.2: Create the ind_cntry DataFrame

# Group by Indicator and Country, and sum the 'Valid' column to count available years.
ind_cntry_grouped = ind_cntry_yr.groupby(['Indicator', 'Country'])['Valid'].sum().reset_index()
ind_cntry_grouped = ind_cntry_grouped.rename(columns={'Valid': 'num_available_years'})

# Merge the grouped data with the criteria DataFrame to get the required number of years for each indicator.
ind_cntry = pd.merge(
    ind_cntry_grouped,
    criteria,
    on='Indicator',
    how='left'
)

# Create the 'Available' column. If the number of available years meets or exceeds
# the criteria, mark as 1, otherwise 0.
ind_cntry['Available'] = (ind_cntry['num_available_years'] >= ind_cntry['number of years']).astype(int)

# Save the resulting DataFrame to an Excel file.
ind_cntry.to_excel("ind_cntry.xlsx", index=False)
print("Saved 'ind_cntry.xlsx'")

In [ ]:
# 2.3: Create the ind_availabilty DataFrame (based on your step 5)

# Calculate the number of unique countries.
total_countries = ind_cntry['Country'].nunique()

# Group by Indicator and calculate the availability percentage.
# This shows what percentage of countries have sufficient data for each indicator.
ind_availabilty = ind_cntry.groupby('Indicator')['Available'].sum().reset_index()
ind_availabilty['ind availabilty'] = (ind_availabilty['Available'] / total_countries) * 100

# Save the resulting DataFrame to an Excel file.
ind_availabilty.to_excel("ind_availabilty.xlsx", index=False)
print("Saved 'ind_availabilty.xlsx'")


# 2.4: Create the cntry_availabilty DataFrame (based on your step 6)

# Calculate the number of unique indicators.
total_indicators = ind_cntry['Indicator'].nunique()

# Group by Country and calculate the availability percentage.
# This shows what percentage of required indicators are available for each country.
cntry_availabilty = ind_cntry.groupby('Country')['Available'].sum().reset_index()
cntry_availabilty['cntry availabilty'] = (cntry_availabilty['Available'] / total_indicators) * 100

# Calculate the average availability across all countries.
region_avg = cntry_availabilty['cntry availabilty'].mean()
cntry_availabilty['region availabilty'] = region_avg

# Save the resulting DataFrame to an Excel file.
cntry_availabilty.to_excel("cntry_availabilty.xlsx", index=False)
print("Saved 'cntry_availabilty.xlsx'")

In [ ]:
# --- Part 3: Generate Charts ---

In [ ]:
# 3.1: Barchart for Overall Country Availability

# --- EDITABLE TITLE FOR THE BARCHART ---
barchart_title = "Overall Data Availability by Country"

# Sort the data for the chart in descending order.
cntry_availabilty_sorted = cntry_availabilty.sort_values('cntry availabilty', ascending=False)

# Set the style and figure size for the plot.
sns.set_style("whitegrid")
plt.figure(figsize=(12, 10))

# Create the horizontal bar plot.
barplot = sns.barplot(
    x='cntry availabilty',
    y='Country',
    data=cntry_availabilty_sorted,
    palette='viridis'
)

# Set the title and labels.
plt.title(barchart_title, fontsize=16)
plt.xlabel("Availability Percentage (%)", fontsize=12)
plt.ylabel("Country", fontsize=12)

# Show the plot.
plt.tight_layout()
plt.savefig("country_availability_barchart.png")
print("Saved 'country_availability_barchart.png'")
plt.show()


# 3.2: Heatmap for a Specific Indicator

# --- EDITABLE INDICATOR AND TITLE FOR THE HEATMAP ---
# Change the indicator_to_plot value to any indicator from your data.
indicator_to_plot = "Population estimates"
heatmap_title = f"Data Availability for: {indicator_to_plot}"

# Filter the data for the selected indicator.
heatmap_data = ind_cntry_yr[ind_cntry_yr['Indicator'] == indicator_to_plot]

# Check if the indicator exists to avoid errors.
if heatmap_data.empty:
    print(f"Error: Indicator '{indicator_to_plot}' not found in the data. Please check the name.")
else:
    # Pivot the data to create a matrix of Country vs. Year.
    heatmap_pivot = heatmap_data.pivot_table(
        index='Country',
        columns='Year',
        values='Valid',
        fill_value=0  # Ensure that any missing combinations are filled with 0.
    )

    # Set the figure size.
    plt.figure(figsize=(18, 10))

    # Create the heatmap.
    # We use a custom colormap: grey for 0 (unavailable) and green for 1 (available).
    # cbar=False hides the color bar, annot=False hides the data values (0s and 1s).
    sns.heatmap(
        heatmap_pivot,
        cmap=ListedColormap(['#d3d3d3', '#2ca02c']), # grey, green
        linewidths=.5,
        linecolor='white',
        cbar=False,
        annot=False
    )

    # Set the title and labels.
    plt.title(heatmap_title, fontsize=16)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Country", fontsize=12)
    plt.xticks(rotation=45) # Rotate x-axis labels for better readability.

    # Show the plot.
    plt.tight_layout()
    plt.savefig("indicator_availability_heatmap.png")
    print("Saved 'indicator_availability_heatmap.png'")
    plt.show()